# Linkage Disequilibrium

**TODO:** How can I plot the LD matrix and such?

In [2]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.6
SparkUI available at http://hms-beagle-5466c684ff-2l8nm:4046
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.58-3f304aae6ce2
LOGGING: writing to /home/olavur/experiments/2020-11-13_fargen1_exome_analysis/fargen-1-exome/notebooks/hail-20201203-1449-0.2.58-3f304aae6ce2.log


In [3]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
from bokeh.models.scales import LogScale
output_notebook()

Loading BokehJS ...

In [6]:
mt = hl.read_matrix_table('/home/olavur/experiments/2020-11-13_fargen1_exome_analysis/data/mt/filtered.mt')

## LD pruning

Remove multi-allelic sites. 

**NOTE:** it's possible to perform HWE tests on all alleles in multi-allelic variants by splitting the variants using [split_multi()](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.split_multi).

In [8]:
mt_diallelic = mt.filter_rows(hl.len(mt.alleles) == 2)

Perform LD pruning.

In [9]:
pruned_variant_table = hl.ld_prune(mt_diallelic.GT, r2=0.2, bp_window_size=500000)
mt_ld_pruned = mt_diallelic.filter_rows(hl.is_defined(pruned_variant_table[mt_diallelic.row_key]))

2020-12-03 14:51:45 Hail: INFO: ld_prune: running local pruning stage with max queue size of 1082402 variants
2020-12-03 14:51:55 Hail: INFO: wrote table with 54790 rows in 5 partitions to /tmp/TWLilQwnj1QYO9en81EfX4
    Total size: 1.12 MiB
    * Rows: 1.12 MiB
    * Globals: 11.00 B
    * Smallest partition: 10627 rows (222.48 KiB)
    * Largest partition:  11245 rows (238.40 KiB)
2020-12-03 14:52:08 Hail: INFO: Wrote all 14 blocks of 54790 x 48 matrix with block size 4096.
2020-12-03 14:56:52 Hail: INFO: wrote table with 57 rows in 27 partitions to /tmp/jxMMJk7qLMVDfcL8jNGOCf
    Total size: 431.71 KiB
    * Rows: 1.55 KiB
    * Globals: 430.16 KiB
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  13 rows (183.00 B)


Write LD pruned `MatrixTable` to file.

In [11]:
mt_ld_pruned.write('../../data/mt/ld_pruned_diallelic.mt', overwrite=True)

2020-12-03 15:04:14 Hail: INFO: wrote matrix table with 54738 rows and 48 columns in 5 partitions to ../../data/mt/ld_pruned_diallelic.mt
    Total size: 35.33 MiB
    * Rows/entries: 35.33 MiB
    * Columns: 221.00 B
    * Globals: 11.00 B
    * Smallest partition: 10617 rows (6.82 MiB)
    * Largest partition:  11235 rows (7.24 MiB)


In [10]:
mt_ld_pruned.count()

(54738, 48)